In [15]:
import pandas as pd
import refrom cache_loader import PersistentCache

In [16]:
# Read mapping from Excel
mapping_url = "https://github.com/srinivashj1/code/raw/main/Select%20Queries%201.xlsx"
mapping_df = pd.read_excel(mapping_url, sheet_name="CRM_S4_TABLE_FIELDS_MAPPING")

# Build table and field mapping dictionaries using correct column names
table_mapping = dict(zip(mapping_df['CRM_TABNAME'], mapping_df['S4_TABNAME']))
field_mapping = dict(zip(mapping_df['CRM_FIELDNAME'], mapping_df['S4_FIELDNAME']))

In [17]:
def transform_query_with_conditional_join_removal(source_query, mapping_df):
    query = re.sub(r'\s+', ' ', source_query.strip())

    # 1. Detect if SINGLE is present
    single = bool(re.search(r'\bSINGLE\b', query, re.IGNORECASE))

    # 2. Detect FOR ALL ENTRIES
    fae_match = re.search(r'FOR ALL ENTRIES IN (\w+)', query, re.IGNORECASE)
    fae_clause = ''
    if fae_match:
        fae_clause = f'FOR ALL ENTRIES IN {fae_match.group(1).upper()}'

    # 3. Detect UP TO ... ROWS
    up_to_match = re.search(r'UP TO (\w+) ROWS', query, re.IGNORECASE)
    up_to_clause = ''
    if up_to_match:
        up_to_clause = f'UP TO {up_to_match.group(1)} ROWS'

    # 4. Remove INNER JOIN for CRMD_LINK and for tables with Remove Join == 'X'
    # Find all INNER JOINs
    join_pattern = r'INNER JOIN (\w+) AS (\w+) ON ([^I]+?)(?=INNER JOIN|\bWHERE\b|\bINTO\b|$)'
    joins = re.findall(join_pattern, query, re.IGNORECASE)
    remove_tables = set()
    for table, alias, _ in joins:
        table_upper = table.upper()
        # Remove CRMD_LINK always
        if table_upper == 'CRMD_LINK':
            remove_tables.add(table_upper)
        # Remove if Remove Join == 'X' in mapping_df
        mapping_row = mapping_df[mapping_df['CRM_TABNAME'].str.upper() == table_upper]
        if not mapping_row.empty and 'Remove Join' in mapping_row.columns:
            if (mapping_row['Remove Join'] == 'X').any():
                remove_tables.add(table_upper)

    # Remove the joins from the query
    def join_replacer(match):
        table = match.group(1).upper()
        if table in remove_tables:
            return ''
        return match.group(0)
    query = re.sub(join_pattern, join_replacer, query, flags=re.IGNORECASE)

    # 5. Extract fields and their table aliases
    fields = []
    select_fields_match = re.search(r'SELECT (.*?) FROM', query, re.IGNORECASE)
    if select_fields_match:
        fields_str = select_fields_match.group(1).strip()
        if ',' in fields_str:
            fields = [f.strip() for f in fields_str.split(',')]
        else:
            fields = [f.strip() for f in fields_str.split() if f.strip()]
    else:
        fields_match = re.search(r'FIELDS (.*?)(WHERE|INTO|$)', query, re.IGNORECASE)
        if fields_match:
            fields_str = fields_match.group(1).strip().rstrip(',')
            if ',' in fields_str:
                fields = [f.strip() for f in fields_str.split(',')]
            else:
                fields = [f.strip() for f in fields_str.split() if f.strip()]

    fields = [f for f in fields if f.strip().upper() != 'SINGLE']

    # 6. Extract table aliases from FROM/JOIN clauses
    alias_table_map = {}
    for match in re.finditer(r'FROM (\w+)(?: AS (\w+))?', query, re.IGNORECASE):
        table = match.group(1).upper()
        alias = match.group(2).upper() if match.group(2) else table
        alias_table_map[alias] = table
    for match in re.finditer(r'JOIN (\w+)(?: AS (\w+))?', query, re.IGNORECASE):
        table = match.group(1).upper()
        alias = match.group(2).upper() if match.group(2) else table
        alias_table_map[alias] = table

    # 7. Transform fields using mapping with table context
    transformed_fields = []
    for field in fields:
        field_for_map = re.sub(r'@DATA\((.*?)\)', r'\1', field)
        field_for_map = field_for_map.lstrip('@')
        if '~' in field_for_map:
            alias, fname = field_for_map.split('~', 1)
            crm_table = alias_table_map.get(alias.upper(), alias.upper())
        else:
            fname = field_for_map
            crm_table = list(alias_table_map.values())[0] if alias_table_map else ''
        fname = fname.strip().upper()
        mapping_row = mapping_df[(mapping_df['CRM_TABNAME'].str.upper() == crm_table) &
                                 (mapping_df['CRM_FIELDNAME'].str.upper() == fname)]
        if not mapping_row.empty:
            s4_field = mapping_row.iloc[0]['S4_FIELDNAME']
        else:
            s4_field = fname
        transformed_fields.append(f'{s4_field} AS {fname}')

    # 8. Extract INTO clause
    into_clause = ''
    into_match = re.search(r'INTO\s+(TABLE\s+)?((@?DATA\([^)]+\))|(@?\w+))', query, re.IGNORECASE)
    if into_match:
        into_var = into_match.group(2)
        if into_match.group(1):
            into_clause = f'INTO TABLE {into_var}'
        else:
            into_clause = f'INTO {into_var}'

    # 9. Extract WHERE clause, map fields with table context
    where_clause = ''
    where_match = re.search(r'WHERE (.*?)(FOR ALL ENTRIES|INTO|UP TO|$)', query, re.IGNORECASE)
    if where_match:
        where_cond = where_match.group(1).strip()
        def replace_tilde(m):
            alias = m.group(1).upper()
            fname = m.group(2).upper()
            crm_table = alias_table_map.get(alias, alias)
            mapping_row = mapping_df[(mapping_df['CRM_TABNAME'].str.upper() == crm_table) &
                                     (mapping_df['CRM_FIELDNAME'].str.upper() == fname)]
            if not mapping_row.empty:
                s4_field = mapping_row.iloc[0]['S4_FIELDNAME']
            else:
                s4_field = fname
            return s4_field
        where_cond = re.sub(r'(\w+)~(\w+)', replace_tilde, where_cond)
        where_clause = f'WHERE {where_cond}'

    # 10. Map main table name
    main_table_alias = list(alias_table_map.keys())[0] if alias_table_map else ''
    main_crm_table = alias_table_map.get(main_table_alias, '')
    mapping_row = mapping_df[mapping_df['CRM_TABNAME'].str.upper() == main_crm_table]
    s4_table = mapping_row.iloc[0]['S4_TABNAME'] if not mapping_row.empty else main_crm_table

    # 11. Compose the final query
    parts = []
    if single:
        parts.append('SELECT SINGLE')
    else:
        parts.append('SELECT')
    parts.append(', '.join(transformed_fields))
    if s4_table:
        parts.append(f'FROM {s4_table}')
    if into_clause:
        parts.append(into_clause)
    if fae_clause:
        parts.append(fae_clause)
    if up_to_clause:
        parts.append(up_to_clause)
    if where_clause:
        parts.append(where_clause)

    transformed_query = ' '.join(parts)
    return transformed_query

In [18]:
# Revised partner handling function: do not add NUMBER_INT to join when force_where_number_int True

def transform_query_with_partner_handling(source_query, mapping_df, *, single_line: bool = True,
                                           add_number_int_condition: bool = True,
                                           join_priority = ("OBJECT_ID","OBJTYPE_H"),
                                           join_fallback = ("GUID",),
                                           include_number_int_in_join: bool = False,
                                           suppress_select_alias_prefix: bool = False,
                                           auto_item_detection: bool = True,
                                           force_where_number_int: bool = True):
    """
    Transform CRM partner join chain into direct MAIN <-> PARTNER join.

    Key rules:
      - Always collapse CRMD_LINK.
      - Join keys from join_priority then fallback; NUMBER_INT excluded when force_where_number_int=True.
      - If (PARTNER_NO requested) AND ((force_where_number_int) OR (not include_number_int_in_join)) add WHERE B~NUMBER_INT IS INITIAL.
      - include_number_int_in_join=True and force_where_number_int=False => NUMBER_INT equality added to ON instead of WHERE predicate.
      - auto_item_detection: if main table ends with '_I' or ITEM_GUID present, enforce WHERE style unless include_number_int_in_join explicitly True.
      - GUID/HEADER_GUID/ITEM_GUID normalized to alias GUID in projection alias (AS GUID) but physical field name mapped.
      - suppress_select_alias_prefix: remove A~/B~ in SELECT list only.
      - single_line output by default.
    """


    original = source_query
    q_norm = re.sub(r'\s+', ' ', source_query.strip())
    flags = re.IGNORECASE | re.DOTALL

    def get_s4_table(crm_table: str):
        rows = mapping_df[mapping_df['CRM_TABNAME'].str.upper() == crm_table.upper()]
        return rows.iloc[0]['S4_TABNAME'] if not rows.empty else crm_table

    def get_s4_field(crm_table: str, crm_field: str):
        rows = mapping_df[(mapping_df['CRM_TABNAME'].str.upper() == crm_table.upper()) &
                          (mapping_df['CRM_FIELDNAME'].str.upper() == crm_field.upper())]
        return rows.iloc[0]['S4_FIELDNAME'] if not rows.empty else crm_field

    single = bool(re.search(r'\bSELECT\s+SINGLE\b', q_norm, flags))
    fae_match = re.search(r'FOR\s+ALL\s+ENTRIES\s+IN\s+(\w+)', q_norm, flags)
    fae_clause = f"FOR ALL ENTRIES IN {fae_match.group(1)}" if fae_match else ''
    up_to_match = re.search(r'UP\s+TO\s+(\w+)\s+ROWS', q_norm, flags)
    up_to_clause = f"UP TO {up_to_match.group(1)} ROWS" if up_to_match else ''

    alias_table_map = {}
    for m in re.finditer(r'\bFROM\s+([A-Z0-9_]+)(?:\s+(?:AS\s+)?([A-Z0-9_]+))?', q_norm, flags):
        t = m.group(1).upper(); a = (m.group(2) or t).upper(); alias_table_map[a] = t
    for m in re.finditer(r'\bJOIN\s+([A-Z0-9_]+)(?:\s+(?:AS\s+)?([A-Z0-9_]+))?', q_norm, flags):
        t = m.group(1).upper(); a = (m.group(2) or t).upper(); alias_table_map[a] = t

    link_alias = next((a for a,t in alias_table_map.items() if t == 'CRMD_LINK'), None)
    partner_alias_orig = next((a for a,t in alias_table_map.items() if t == 'CRMD_PARTNER'), None)
    main_alias_orig = next((a for a,t in alias_table_map.items() if t not in ('CRMD_LINK','CRMD_PARTNER')), None)

    if not (link_alias and partner_alias_orig and main_alias_orig):
        return re.sub(r'\s+', ' ', original.strip())

    main_crm = alias_table_map[main_alias_orig]
    partner_crm = alias_table_map[partner_alias_orig]

    if auto_item_detection:
        is_item = main_crm.endswith('_I') or bool(re.search(r'ITEM_GUID', q_norm, flags))
        if is_item and not include_number_int_in_join:
            add_number_int_condition = True
            force_where_number_int = True

    main_s4 = get_s4_table(main_crm)
    partner_s4 = get_s4_table(partner_crm)

    into_match = re.search(r'\bINTO\s+TABLE\s+([@A-Z0-9_()]+)|\bINTO\s+([@A-Z0-9_()]+)', q_norm, flags)
    into_clause = ''
    if into_match:
        into_var = into_match.group(1) or into_match.group(2)
        into_clause = f'INTO TABLE {into_var}' if into_match.group(1) else f'INTO {into_var}'

    where_match = re.search(r'\bWHERE\b(.*)$', q_norm, flags)
    where_cond_raw = where_match.group(1).strip() if where_match else ''

    sel_match = re.search(r'\bSELECT\b\s+(?:SINGLE\s+)?(.*?)\s+FROM\s', q_norm, flags)
    if not sel_match:
        return re.sub(r'\s+', ' ', original.strip())
    raw_fields = sel_match.group(1).strip()
    if raw_fields == '*':
        field_tokens = ['A~GUID', f'{partner_alias_orig}~PARTNER_NO']
    else:
        field_tokens = [tok.strip() for tok in (raw_fields.split(',') if ',' in raw_fields else raw_fields.split()) if tok.strip()]

    partner_no_requested = any(tok.upper().endswith('PARTNER_NO') for tok in field_tokens)

    output_fields = []
    for tok in field_tokens:
        tok_clean = re.sub(r'@DATA\((.*?)\)', r'\1', tok).lstrip('@')
        if '~' in tok_clean:
            crm_alias, crm_field = tok_clean.split('~',1)
            crm_alias_up = crm_alias.upper(); crm_field_up = crm_field.upper()
            crm_table = partner_crm if crm_alias_up == partner_alias_orig else main_crm
            s4_field = get_s4_field(crm_table, crm_field_up)
            out_as = 'GUID' if crm_field_up in ('GUID','HEADER_GUID','ITEM_GUID') else crm_field_up
            if suppress_select_alias_prefix:
                output_fields.append(f"{s4_field} AS {out_as}")
            else:
                out_alias = 'B' if crm_alias_up == partner_alias_orig else 'A'
                output_fields.append(f"{out_alias}~{s4_field} AS {out_as}")
        else:
            crm_field_up = tok_clean.upper()
            s4_field = get_s4_field(main_crm, crm_field_up)
            out_as = 'GUID' if crm_field_up in ('GUID','HEADER_GUID','ITEM_GUID') else crm_field_up
            output_fields.append(f"{'' if suppress_select_alias_prefix else 'A~'}{s4_field} AS {out_as}")

    join_parts = []
    exclusion = {'NUMBER_INT','NUMBER_EXT'} if (force_where_number_int or not include_number_int_in_join) else {'NUMBER_EXT'}
    for key in join_priority:
        left = get_s4_field(main_crm, key)
        right = get_s4_field(partner_crm, key)
        if left and right and key.upper() not in exclusion:
            join_parts.append(f"B~{right} = A~{left}")
    if not join_parts:
        for key in join_fallback:
            left = get_s4_field(main_crm, key)
            right = get_s4_field(partner_crm, key)
            if left and right and key.upper() not in exclusion:
                join_parts.append(f"B~{right} = A~{left}")
                break

    if include_number_int_in_join and not force_where_number_int:
        ni_main = get_s4_field(main_crm, 'NUMBER_INT')
        ni_partner = get_s4_field(partner_crm, 'NUMBER_INT')
        if ni_main and ni_partner:
            join_parts.append(f"B~{ni_partner} = A~{ni_main}")

    join_clause = 'INNER JOIN ' + partner_s4 + ' AS B ON ' + ' AND '.join(join_parts) if join_parts else ''

    def map_where_ref(m):
        w_alias = m.group(1).upper(); w_field = m.group(2).upper()
        crm_table = partner_crm if w_alias == partner_alias_orig else main_crm
        out_alias = 'B' if w_alias == partner_alias_orig else 'A'
        return f"{out_alias}~{get_s4_field(crm_table, w_field)}"

    mapped_where = re.sub(r'([A-Z0-9_]+)~([A-Z0-9_]+)', map_where_ref, where_cond_raw) if where_cond_raw else ''

    if (partner_no_requested and add_number_int_condition and (force_where_number_int or not include_number_int_in_join)):
        np_int = get_s4_field(partner_crm, 'NUMBER_INT')
        if np_int and f"B~{np_int} IS INITIAL" not in mapped_where:
            mapped_where = (mapped_where + ' AND ' if mapped_where else '') + f"B~{np_int} IS INITIAL"

    select_clause = ('SELECT SINGLE' if single else 'SELECT') + ' ' + ', '.join(output_fields)
    from_clause = f'FROM {main_s4} AS A'

    parts = [select_clause, from_clause]
    if join_clause: parts.append(join_clause)
    if into_clause: parts.append(into_clause)
    if fae_clause: parts.append(fae_clause)
    if up_to_clause: parts.append(up_to_clause)
    if mapped_where: parts.append('WHERE ' + mapped_where)

    if single_line:
        res = ' '.join(parts)
        if not res.endswith('.'): res += '.'
        return res

    pretty = [select_clause, from_clause]
    if join_clause: pretty.append(join_clause)
    if into_clause: pretty.append(into_clause)
    if fae_clause: pretty.append(fae_clause)
    if up_to_clause: pretty.append(up_to_clause)
    if mapped_where:
        for i, segment in enumerate(re.split(r'\s+AND\s+', mapped_where)):
            pretty.append(('WHERE ' if i==0 else 'AND ') + segment)
    if not pretty[-1].endswith('.'): pretty[-1] += '.'
    return '\n'.join(pretty)

---

In [19]:
def transform_query_auto(source_query, mapping_df):
    """
    Automatically choose the correct transformation function based on the query content.
    If the query contains a join to CRMD_PARTNER, use the partner handling transformation.
    Otherwise, use the conditional join removal transformation.
    """
    q = re.sub(r'\s+', ' ', source_query.strip().upper())
    partner_join = bool(re.search(r'JOIN\s+CRMD_PARTNER', q))
    if partner_join:
        return transform_query_with_partner_handling(source_query, mapping_df)
    else:
        return transform_query_with_conditional_join_removal(source_query, mapping_df)